# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_file = "Output/weather_df.csv"
weather_df = pd.read_csv(data_file)
weather_df.head()

,City Name,Lat,Lng,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Malinyi,-8.93,36.13,76.08,57,28,3.47
1,Vangaindrano,-23.35,47.60,67.01,88,13,4.88
2,San,13.30,-4.90,86.94,42,20,4.18
3,Mataura,-46.19,168.86,42.01,89,100,7.00
4,Port,-7.03,29.76,76.73,66,93,4.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity (%)"]

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather = weather_df.loc[(weather_df["Temperature (F)"] < 80) &
                               (weather_df["Temperature (F)"] > 70) &
                               (weather_df["Humidity (%)"] < 50) &
                               (weather_df["Cloudiness (%)"] == 0), :]
len(ideal_weather)
ideal_weather

,City Name,Lat,Lng,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
12,Nizwá,22.93,57.53,77.61,32,0,1.07
135,Buraidah,26.33,43.98,75.20,31,0,4.70
152,Tigre,-34.43,-58.58,72.82,35,0,10.29
320,Crowley,30.21,-92.37,75.99,45,0,4.00
413,Washington,47.50,-120.50,76.48,42,0,3.49
491,Jacobina,-11.18,-40.52,75.90,46,0,4.00
496,Camargo,27.67,-105.17,76.01,19,0,5.06


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather[["City Name", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""
hotel_df["Hotel Rating"] = ""
hotel_df["Country"] = ""
hotel_df

<ipython-input-6-f57f84ea393b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""
<ipython-input-6-f57f84ea393b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Address"] = ""
<ipython-input-6-f57f84ea393b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

,City Name,Lat,Lng,Hotel Name,Hotel Address,Hotel Rating
12,Nizwá,22.93,57.53,,,
135,Buraidah,26.33,43.98,,,
152,Tigre,-34.43,-58.58,,,
320,Crowley,30.21,-92.37,,,
413,Washington,47.50,-120.50,,,
491,Jacobina,-11.18,-40.52,,,
496,Camargo,27.67,-105.17,,,


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

C:\Users\jakan\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City Name,Lat,Lng,Hotel Name,Hotel Address,Hotel Rating
12,Nizwá,22.93,57.53,Bait Al Aqr,Nizwa,4.8
135,Buraidah,26.33,43.98,Mövenpick Hotel Qassim,"King Khalid Rd, Buraydah",4.2
152,Tigre,-34.43,-58.58,Villa San Gimignano,"3198, DPT, Miguens, Victoria",4.8
320,Crowley,30.21,-92.37,Days Inn by Wyndham Crowley,"9571 Egan Hwy, Crowley",3.8
413,Washington,47.50,-120.50,Cascade Valley Inn,"56 Mountainside Dr, Cashmere",4.9
491,Jacobina,-11.18,-40.52,Hotel Serra do Ouro,"Largo Monte Tabor, s/n - Caixa D'água, Jacobina",4.2
496,Camargo,27.67,-105.17,Hotel Santa Fe,"Carretera Panamericana Km. 67 Salida Sur, Revo...",4.6


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))